In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os
from datetime import datetime

# 1. Configure streams
pipeline = rs.pipeline()
config   = rs.config()

# Set resolution and frame rate
COLOR_WIDTH  = 640
COLOR_HEIGHT = 480
DEPTH_WIDTH  = 640
DEPTH_HEIGHT = 480
FPS          = 30

config.enable_stream(rs.stream.color, COLOR_WIDTH, COLOR_HEIGHT, rs.format.bgr8, FPS)
config.enable_stream(rs.stream.depth, DEPTH_WIDTH, DEPTH_HEIGHT, rs.format.z16, FPS)

# 2. Start streaming
profile = pipeline.start(config)

# 3. Align depth to color (so depth & RGB overlap pixel-wise)
align_to = rs.stream.color
align     = rs.align(align_to)

# 4. Create output directories
rgb_dir   = "Dataset/mix/rgb"
depth_dir = "Dataset/mix/depth"
os.makedirs(rgb_dir,   exist_ok=True)
os.makedirs(depth_dir, exist_ok=True)

print("Press 'c' to capture a frame, 'q' to quit.")

try:
    while True:
        # 5. Wait for a pair of frames
        frames = pipeline.wait_for_frames()
        aligned = align.process(frames)

        depth_frame = aligned.get_depth_frame()
        color_frame = aligned.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # 6. Convert to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())  # uint16
        color_image = np.asanyarray(color_frame.get_data())  # uint8

        # 7. Show live preview
        depth_vis = cv2.convertScaleAbs(depth_image, alpha=0.03)  # quick 8-bit preview
        depth_vis = cv2.applyColorMap(depth_vis, cv2.COLORMAP_JET)
        cv2.imshow("RGB",   color_image)
        cv2.imshow("Depth preview", depth_vis)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
            rgb_path   = os.path.join(rgb_dir,   f"{timestamp}.png")
            depth_path = os.path.join(depth_dir, f"{timestamp}.png")

            # 8. Save images
            cv2.imwrite(rgb_path,   color_image)   # 8-bit RGB PNG
            # depth_image is uint16, so OpenCV will encode it as 16-bit PNG automatically
            cv2.imwrite(depth_path, depth_image)

            print(f"Saved RGB → {rgb_path}")
            print(f"Saved Depth → {depth_path}")

        elif key == ord('q'):
            break

finally:
    # 9. Clean up
    pipeline.stop()
    cv2.destroyAllWindows()


In [3]:

import pyrealsense2 as rs
import numpy as np
import cv2
import os
import datetime

# Create output directories
base_dir = 'recordings'
rgb_dir = os.path.join(base_dir, 'rgb')
depth_dir = os.path.join(base_dir, 'depth')
ir_dir = os.path.join(base_dir, 'ir')
os.makedirs(rgb_dir, exist_ok=True)
os.makedirs(depth_dir, exist_ok=True)
os.makedirs(ir_dir, exist_ok=True)

# Configure RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.infrared, 1, 640, 480, rs.format.y8, 30)

# Start streaming
pipeline.start(config)

recording = False
rgb_writer = None
depth_writer = None
ir_writer = None

print("Press 'c' to start recording, 'e' to end recording, 'q' to quit.")

try:
    while True:
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        depth_frame = frames.get_depth_frame()
        ir_frame = frames.get_infrared_frame(1)
        if not color_frame or not depth_frame or not ir_frame:
            continue

        # Convert to numpy arrays
        color_image = np.asanyarray(color_frame.get_data())
        depth_image = np.asanyarray(depth_frame.get_data())
        ir_image = np.asanyarray(ir_frame.get_data())

        # Visualization conversions
        depth_colormap = cv2.applyColorMap(
            cv2.convertScaleAbs(depth_image, alpha=0.03),
            cv2.COLORMAP_JET)
        ir_colormap = cv2.applyColorMap(
            cv2.convertScaleAbs(ir_image, alpha=0.03),
            cv2.COLORMAP_JET)

        # Display streams
        cv2.imshow('RGB Stream', color_image)
        cv2.imshow('Depth Stream', depth_colormap)
        cv2.imshow('IR Stream', ir_colormap)

        key = cv2.waitKey(1) & 0xFF

        if key == ord('c') and not recording:
            # Start recording
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            rgb_path = os.path.join(rgb_dir, f"rgb_{timestamp}.avi")
            depth_path = os.path.join(depth_dir, f"depth_{timestamp}.avi")
            ir_path = os.path.join(ir_dir, f"ir_{timestamp}.avi")
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            fps = 30
            height, width, _ = color_image.shape
            rgb_writer = cv2.VideoWriter(rgb_path, fourcc, fps, (width, height))
            depth_writer = cv2.VideoWriter(depth_path, fourcc, fps, (width, height))
            ir_writer = cv2.VideoWriter(ir_path, fourcc, fps, (width, height))
            recording = True
            print(f"Recording started:\n RGB -> {rgb_path}\n Depth -> {depth_path}\n IR -> {ir_path}")

        elif key == ord('e') and recording:
            # Stop recording
            recording = False
            rgb_writer.release()
            depth_writer.release()
            ir_writer.release()
            rgb_writer = depth_writer = ir_writer = None
            print("Recording stopped.")

        elif key == ord('q'):
            # Quit application
            break

        # Write frames if recording
        if recording:
            rgb_writer.write(color_image)
            depth_writer.write(depth_colormap)
            ir_writer.write(ir_colormap)

finally:
    # Cleanup
    if recording and rgb_writer and depth_writer and ir_writer:
        rgb_writer.release()
        depth_writer.release()
        ir_writer.release()
    pipeline.stop()
    cv2.destroyAllWindows()


Press 'c' to start recording, 'e' to end recording, 'q' to quit.
Recording started:
 RGB -> recordings\rgb\rgb_20250723_114322.avi
 Depth -> recordings\depth\depth_20250723_114322.avi
 IR -> recordings\ir\ir_20250723_114322.avi
Recording stopped.


In [4]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os
import datetime

# Base recordings directory
base_dir = 'recordings'
os.makedirs(base_dir, exist_ok=True)

# Configure RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color,    640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth,    640, 480, rs.format.z16,   30)
config.enable_stream(rs.stream.infrared, 1, 640, 480, rs.format.y8,   30)
profile = pipeline.start(config)

# Align depth to color
align = rs.align(rs.stream.color)

recording   = False
session_dir = None
frame_idx   = 0

print("Press 'c' to start recording, 'e' to end recording, 'q' to quit.")

try:
    while True:
        # Get aligned frames
        frames      = pipeline.wait_for_frames()
        aligned     = align.process(frames)
        color_frame = aligned.get_color_frame()
        depth_frame = aligned.get_depth_frame()
        ir_frame    = aligned.get_infrared_frame(1)

        if not color_frame or not depth_frame or not ir_frame:
            continue

        # Raw data arrays
        color_image = np.asanyarray(color_frame.get_data())
        depth_image = np.asanyarray(depth_frame.get_data())
        ir_image    = np.asanyarray(ir_frame.get_data())

        # For display only
        depth_vis = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        ir_vis    = cv2.applyColorMap(cv2.convertScaleAbs(ir_image, alpha=0.03), cv2.COLORMAP_BONE)

        cv2.imshow('RGB Stream',   color_image)
        cv2.imshow('Depth Stream', depth_vis)
        cv2.imshow('IR Stream',    ir_vis)

        key = cv2.waitKey(1) & 0xFF

        # Start recording
        if key == ord('c') and not recording:
            ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            session_dir = os.path.join(base_dir, f"session_{ts}")
            rgb_dir     = os.path.join(session_dir, 'rgb')
            depth_dir   = os.path.join(session_dir, 'depth')
            ir_dir      = os.path.join(session_dir, 'ir')
            os.makedirs(rgb_dir, exist_ok=True)
            os.makedirs(depth_dir, exist_ok=True)
            os.makedirs(ir_dir, exist_ok=True)
            recording = True
            frame_idx = 0
            print(f"Recording started. Saving to {session_dir}")

        # Stop recording
        elif key == ord('e') and recording:
            recording = False
            print(f"Recording stopped. {frame_idx} frames saved in each folder.")

        # Quit
        elif key == ord('q'):
            break

        # Save raw frames if recording
        if recording:
            rgb_path   = os.path.join(rgb_dir,   f"rgb_{frame_idx:06d}.png")
            depth_path = os.path.join(depth_dir, f"depth_{frame_idx:06d}.png")
            ir_path    = os.path.join(ir_dir,    f"ir_{frame_idx:06d}.png")

            cv2.imwrite(rgb_path,   color_image)  # 8-bit BGR
            cv2.imwrite(depth_path, depth_image)  # 16-bit PNG with original depth
            cv2.imwrite(ir_path,    ir_image)     # 8-bit PNG with original IR

            frame_idx += 1

finally:
    pipeline.stop()
    cv2.destroyAllWindows()


Press 'c' to start recording, 'e' to end recording, 'q' to quit.
Recording started. Saving to recordings\session_20250723_114622
Recording stopped. 104 frames saved in each folder.
